<a href="https://colab.research.google.com/github/pranav1246/Tds-project-1/blob/main/resume_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas numpy torch transformers pdfplumber spacy nltk datasets fastapi streamlit
!pip install scikit-learn matplotlib seaborn
!pip install pytorch-lightning
!python -m spacy download en_core_web_sm


     --------------------------------------- 12.8/12.8 MB 11.0 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
done


In [4]:
import pandas as pd
df = pd.read_csv("01_people.csv")
df.head()



,person_id,name,email,phone,linkedin
0,1,Database Administrator,NaN,NaN,NaN
1,2,Database Administrator,NaN,NaN,NaN
2,3,Oracle Database Administrator,NaN,NaN,NaN
3,4,Amazon Redshift Administrator and ETL Develope...,NaN,NaN,NaN
4,5,Scrum Master Scrum Master Scrum Master,NaN,NaN,NaN


In [5]:
import json
with open("fixed.json", "r", encoding="utf-8") as file:
    ner_data = json.load(file)
print(ner_data[0])


{'content': "Abhishek Jha\nApplication Development Associate - Accenture\n\nBengaluru, Karnataka - Email me on Indeed: indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a\n\n• To work for an organization which provides me the opportunity to improve my skills\nand knowledge for my individual and company's growth in best possible ways.\n\nWilling to relocate to: Bangalore, Karnataka\n\nWORK EXPERIENCE\n\nApplication Development Associate\n\nAccenture -\n\nNovember 2017 to Present\n\nRole: Currently working on Chat-bot. Developing Backend Oracle PeopleSoft Queries\nfor the Bot which will be triggered based on given input. Also, Training the bot for different possible\nutterances (Both positive and negative), which will be given as\ninput by the user.\n\nEDUCATION\n\nB.E in Information science and engineering\n\nB.v.b college of engineering and technology -  Hubli, Karnataka\n\nAugust 2013 to June 2017\n\n12th in Mathematics\n\nWoodbine modern school\n\nApril 2011 to March 2013\n\n10th\n\nKendriya 

In [6]:
import re
import nltk
import pandas as pd
from nltk.corpus import stopwords


nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

def clean_text(text):
    if not isinstance(text, str):
        return ""

    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    text = " ".join([word for word in text.split() if word not in stop_words])
    return text


df["cleaned_name"] = df["name"].apply(clean_text)


print(df)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


       person_id                                               name email  \
0              1                             Database Administrator   NaN   
1              2                             Database Administrator   NaN   
2              3                      Oracle Database Administrator   NaN   
3              4  Amazon Redshift Administrator and ETL Develope...   NaN   
4              5             Scrum Master Scrum Master Scrum Master   NaN   
...          ...                                                ...   ...   
54928      54929                              Lead Python Developer   NaN   
54929      54930                        Full Stack Python Developer   NaN   
54930      54931                                          Eli Lilly   NaN   
54931      54932                                   Python Developer   NaN   
54932      54933                                         Job Seeker   NaN   

      phone linkedin                                       cleaned_name  
0

In [7]:
ner_samples = []

for entry in ner_data:
    text = entry["content"]

    if "annotation" in entry and entry["annotation"]:
        labels = {
            label["label"][0]: label["points"][0]["text"]
            for label in entry["annotation"] if label["label"] and label["points"]
        }
    else:
        labels = {}

    ner_samples.append((text, labels))

print(ner_samples[:5])

[("Abhishek Jha\nApplication Development Associate - Accenture\n\nBengaluru, Karnataka - Email me on Indeed: indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a\n\n• To work for an organization which provides me the opportunity to improve my skills\nand knowledge for my individual and company's growth in best possible ways.\n\nWilling to relocate to: Bangalore, Karnataka\n\nWORK EXPERIENCE\n\nApplication Development Associate\n\nAccenture -\n\nNovember 2017 to Present\n\nRole: Currently working on Chat-bot. Developing Backend Oracle PeopleSoft Queries\nfor the Bot which will be triggered based on given input. Also, Training the bot for different possible\nutterances (Both positive and negative), which will be given as\ninput by the user.\n\nEDUCATION\n\nB.E in Information science and engineering\n\nB.v.b college of engineering and technology -  Hubli, Karnataka\n\nAugust 2013 to June 2017\n\n12th in Mathematics\n\nWoodbine modern school\n\nApril 2011 to March 2013\n\n10th\n\nKendriya Vidyalaya\

In [8]:
from sklearn.preprocessing import LabelEncoder


label_encoder = LabelEncoder()
df["category_encoded"] = label_encoder.fit_transform(df["name"])


print(label_encoder.classes_)


['(IT Tech Lead / Delivery Manager Consultant), Va'
 '10g Database Administrator' '25B Information Technology Specialist' ...
 'undisclosed' 'vCIO - Virtual Chief Information Officer' nan]


In [11]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification


tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


def tokenize_data(texts, labels, tokenizer, max_length=128):
    tokens = tokenizer(
        texts.tolist(),
        max_length=max_length,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )
    labels = torch.tensor(labels)
    return tokens, labels

tokens, labels = tokenize_data(df["cleaned_name"], df["category_encoded"], tokenizer)


In [12]:
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam
import torch.nn.functional as F


dataset = TensorDataset(tokens["input_ids"], tokens["attention_mask"], labels)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)


model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_encoder.classes_))


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = Adam(model.parameters(), lr=2e-5)

for epoch in range(3):
    model.train()
    for batch in dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1} Loss: {loss.item()}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


KeyboardInterrupt: 

In [ ]:
from transformers import AutoModelForTokenClassification, AutoTokenizer, pipeline

# Load BERT-based NER Model
ner_model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
ner_tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")

# Create NER Pipeline
ner_pipeline = pipeline("ner", model=ner_model, tokenizer=ner_tokenizer)

# Test on sample resume text
resume_text = "Abhishek Jha is an Application Development Associate at Accenture with experience in Java, Python, and Machine Learning."
ner_results = ner_pipeline(resume_text)

# Print extracted entities
for entity in ner_results:
    print(f"{entity['word']} → {entity['entity']}")


In [24]:
import pdfplumber

def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + " "
    return text.strip()

# Example usage
resume_text = extract_text_from_pdf("PranavN_Analyst.pdf")
print(resume_text)


Pranav N
Bengaluru•+916361135685•pranavn0406@gmail.com•linkedin.com/in/pranavn46
Results-driven Data Analyst with expertise in Python, Machine Learning and Business Intelligence. Profi
cient in statistical methods, predictive modeling, and data visualization to drive data-driven decision-
making.
WORK EXPERIENCE
Talview 08/2023 - 01/2024
Software Intern•Internship Bengaluru
• Developed a comprehensive Slack bot for Talview employees to streamline leave management by
integrating with Zoho People.
• Utilized Zoho People APIs to fetch and update leave data.
• Implemented Redis to store access tokens securely.
• Developed the Slack bot from scratch using Node.js, mapping employee IDsto Slack IDs using
PostgreSQL for effi cient data management.
• This solution signifi cantly simplifi ed the leave application process foremployees, allowing them to
apply for leaves directly from Slack.
EDUCATION
Bachelor of Science in Programming & Data Science
Indian Institute of Technology (IIT) Madras•GPA:

In [ ]:
!streamlit run app.py & ./cloudflared tunnel --url http://localhost:8501


2025-03-12T04:35:59Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-03-12T04:35:59Z INF Requesting new quick Tunnel on trycloudflare.com...



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.237.26.11:8501

2025-03-12T04:36:03Z INF +--------------------------------------------------------------------------------------------+
2025-03-12T04:36:03Z INF |

In [30]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.9 MB/s eta 0:00:00
